In [1]:
# T# ====================================================
# VETERINARY ANIMAL CLASSIFICATION WITH TRANSFORMERS
# Dataset: Animals-10 (Kaggle)
# Model: Vision Transformer (ViT) - No Pretrained Weights
# Accuracy Target: >85%
# Domain: Veterinary (Animal Health & Species Identification)
# SDG Alignment: Life on Land (SDG 15)
# ====================================================

# Install required packages
!pip install torch torchvision matplotlib seaborn scikit-learn tqdm pillow pandas -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

   
            
       

    
   

Using device: cuda
GPU available: True
GPU: Tesla P100-PCIE-16GB


In [2]:
# ====================================================
# VETERINARY ANIMAL CLASSIFICATION - FIXED VERSION
# Dataset: Animals-10 (Kaggle)
# Model: Custom Vision Transformer (No Pretrained Weights)
# ====================================================

# Install packages
!pip install torch torchvision matplotlib seaborn scikit-learn tqdm pillow pandas -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ====================================================
# 1. LOAD DATASET (FIXED - NO SHELL COMMANDS IN PYTHON INDENTATION)
# ====================================================

print("="*50)
print("LOADING ANIMALS-10 DATASET")
print("="*50)

# Dataset paths to try
dataset_paths = [
    "/kaggle/input/animals10/animals10",
    "/kaggle/input/animals10",
    "/kaggle/input/animals-10",
    "/kaggle/input/animal10"
]

data_dir = None
for path in dataset_paths:
    if os.path.exists(path):
        data_dir = path
        print(f"✓ Found dataset at: {path}")
        break

# If not found, show instructions
if data_dir is None:
    print("\n❌ Dataset not found.")
    print("\n📋 PLEASE ADD THIS DATASET:")
    print("1. Click '+ Add Data' button on Kaggle")
    print("2. Search for 'animals10'")
    print("3. Use: https://www.kaggle.com/datasets/alessiocorrado99/animals10")
    print("\nUsing synthetic data for demo...")
    # Create demo data
    from torchvision.datasets import FakeData
    full_dataset = FakeData(size=2000, image_size=(3, 224, 224), num_classes=10, 
                           transform=transforms.ToTensor())
else:
    # Load real dataset
    print(f"\nLoading from: {data_dir}")
    
    # Check what's in the directory (using Python, not shell commands)
    print("\nExploring directory structure...")
    try:
        items = os.listdir(data_dir)
        print(f"Found {len(items)} items in directory")
        
        # Show first 10 items
        for i, item in enumerate(items[:10]):
            item_path = os.path.join(data_dir, item)
            if os.path.isdir(item_path):
                sub_items = os.listdir(item_path)[:3]
                print(f"  {item}/ [directory] - sample: {sub_items}")
            else:
                print(f"  {item} [file]")
        
        # Load with ImageFolder
        print("\nLoading images...")
        train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
        full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
        print(f"✓ Loaded {len(full_dataset)} images")
        print(f"✓ Classes: {len(full_dataset.classes)}")
        print(f"✓ Class names: {full_dataset.classes}")
        
        # Show class distribution
        print("\nClass distribution:")
        class_counts = {}
        for _, label in full_dataset.samples:
            class_name = full_dataset.classes[label]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        for cls, count in class_counts.items():
            print(f"  {cls}: {count} images")
            
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Using synthetic data instead...")
        from torchvision.datasets import FakeData
        full_dataset = FakeData(size=2000, image_size=(3, 224, 224), num_classes=10, 
                               transform=transforms.ToTensor())


 


Device: cuda
GPU: Tesla P100-PCIE-16GB
LOADING ANIMALS-10 DATASET
✓ Found dataset at: /kaggle/input/animals10

Loading from: /kaggle/input/animals10

Exploring directory structure...
Found 2 items in directory
  translate.py [file]
  raw-img/ [directory] - sample: ['cavallo', 'pecora', 'elefante']

Loading images...
✓ Loaded 26179 images
✓ Classes: 1
✓ Class names: ['raw-img']

Class distribution:
  raw-img: 26179 images
